In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()

for sub in newdemo['age_at_cnb']:
    for task in ['rest', 'nback', 'emoid']:
        f = f'{newdir}/fc/{sub}_task-{task}_fc.npy'
        if not Path(f).exists():
            continue
        p = np.load(f)
        newfc[f'{sub}-{task}'] = p

print(len(newfc))

4343


In [2]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=False):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [21]:
nepochs = 2000
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(2):
        basis = AngleBasis(1)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.6672155261039734
501 0.5726746320724487
1001 0.554268479347229
1501 0.5538817048072815
1999 0.5538817048072815
Saving
1 0.6465706825256348
501 0.5491579174995422
1001 0.5487358570098877
1501 0.5487358570098877
1999 0.5487358570098877
Saving
Finished 1 600001103037-rest
1 0.6669191122055054
501 0.5971413850784302
1001 0.5745021104812622
1501 0.5744550824165344
1999 0.5744550824165344
Saving
1 0.6636590957641602
501 0.5760312676429749
1001 0.5738241672515869
1501 0.5738241672515869
1999 0.5738241672515869
Saving
Finished 2 600001103037-nback
1 0.6877912878990173
501 0.5986784100532532
1001 0.574433445930481
1501 0.5744328498840332
1999 0.5739299058914185
Saving
1 0.6891675591468811
501 0.5757912397384644
1001 0.575369656085968
1501 0.575369656085968
1999 0.575369656085968
Finished 3 600001103037-emoid
1 0.6205719709396362
501 0.49304234981536865
1001 0.4751826524734497
1501 0.4743094742298126
1999 0.4730075001716614
Saving
1 0.6266458034515381
501 0.49886399507522583
1001 0.465890169

501 0.5185617208480835
1001 0.5009622573852539
1501 0.5007349252700806
1999 0.5007334351539612
Saving
1 0.6337714791297913
501 0.49124443531036377
1001 0.4912271201610565
1501 0.49120453000068665
1999 0.49120453000068665
Saving
Finished 31 600045443700-rest
1 0.6493714451789856
501 0.5271182060241699
1001 0.524008572101593
1501 0.524008572101593
1999 0.524008572101593
Saving
1 0.6457521319389343
501 0.5264618396759033
1001 0.5230795741081238
1501 0.523079514503479
1999 0.523079514503479
Saving
Finished 32 600045443700-nback
1 0.6575635075569153
501 0.5575845241546631
1001 0.5354941487312317
1501 0.5329989194869995
1999 0.5329989194869995
Saving
1 0.6479565501213074
501 0.5451455116271973
1001 0.5410656332969666
1501 0.540875256061554
1999 0.540875256061554
Finished 33 600045443700-emoid
1 0.6307671666145325
501 0.3559451401233673
1001 0.3535274565219879
1501 0.3535274565219879
1999 0.3535274565219879
Saving
1 0.6565278172492981
501 0.37488114833831787
1001 0.36832180619239807
1501 0.36

501 0.551005482673645
1001 0.5468464493751526
1501 0.5468167662620544
1999 0.5468167662620544
Saving
1 0.6533893942832947
501 0.5482902526855469
1001 0.5420185327529907
1501 0.540908932685852
1999 0.5407089591026306
Saving
Finished 61 600085654611-nback
1 0.6499772071838379
501 0.5691289305686951
1001 0.5674019455909729
1501 0.5674018859863281
1999 0.5674018263816833
Saving
1 0.6582557559013367
501 0.5702003240585327
1001 0.5674420595169067
1501 0.5650283694267273
1999 0.5650283694267273
Saving
Finished 62 600085654611-emoid
1 0.597315788269043
501 0.43516066670417786
1001 0.42938074469566345
1501 0.4292362332344055
1999 0.42923611402511597
Saving
1 0.6447468996047974
501 0.4434606730937958
1001 0.436428964138031
1501 0.4345562756061554
1999 0.4345162808895111
Finished 63 600090997431-nback
1 0.6194884181022644
501 0.473215252161026
1001 0.47063109278678894
1501 0.4697703719139099
1999 0.4677713215351105
Saving
1 0.6301841735839844
501 0.4767540991306305
1001 0.4595726728439331
1501 0.

501 0.3827538788318634
1001 0.379474014043808
1501 0.37947404384613037
1999 0.37947404384613037
Saving
1 0.5997864603996277
501 0.3683525323867798
1001 0.36815106868743896
1501 0.3681360185146332
1999 0.3681360185146332
Saving
Finished 91 600135180939-rest
1 0.5936850905418396
501 0.3920336365699768
1001 0.3909274935722351
1501 0.39092743396759033
1999 0.39092743396759033
Saving
1 0.6162481307983398
501 0.39674049615859985
1001 0.3953253924846649
1501 0.39506906270980835
1999 0.39506906270980835
Finished 92 600135180939-nback
1 0.6246843934059143
501 0.401275634765625
1001 0.40056732296943665
1501 0.40056735277175903
1999 0.40056735277175903
Saving
1 0.6259762048721313
501 0.3941728174686432
1001 0.39336368441581726
1501 0.3933635950088501
1999 0.3933635950088501
Saving
Finished 93 600135180939-emoid
1 0.6463869214057922
501 0.48870718479156494
1001 0.4836796820163727
1501 0.4831449091434479
1999 0.48296594619750977
Saving
1 0.6350953578948975
501 0.5067578554153442
1001 0.488854110240

501 0.44317182898521423
1001 0.44302135705947876
1501 0.4430212378501892
1999 0.4430212378501892
Saving
1 0.631450891494751
501 0.4510194659233093
1001 0.44013330340385437
1501 0.43811702728271484
1999 0.43811702728271484
Saving
Finished 121 600201270162-nback
1 0.6858949065208435
501 0.41742607951164246
1001 0.417401522397995
1501 0.417401522397995
1999 0.417401522397995
Saving
1 0.6815400123596191
501 0.4174148738384247
1001 0.4174015522003174
1501 0.417401522397995
1999 0.417401522397995
Finished 122 600201270162-emoid
1 0.6034316420555115
501 0.40135353803634644
1001 0.3934955298900604
1501 0.3926088809967041
1999 0.39260223507881165
Saving
1 0.5882569551467896
501 0.4071377217769623
1001 0.40627220273017883
1501 0.404328852891922
1999 0.40347591042518616
Finished 123 600209790043-nback
1 0.6018296480178833
501 0.30383798480033875
1001 0.2966139316558838
1501 0.29657718539237976
1999 0.29657700657844543
Saving
1 0.5917639136314392
501 0.3039649426937103
1001 0.29855117201805115
150

...

501 0.5635635256767273
1001 0.5421251654624939
1501 0.5420645475387573
1999 0.5420645475387573
Saving
Finished 4196 609679707006-emoid
1 0.6832719445228577
501 0.5657253265380859
1001 0.5639185309410095
1501 0.5639185309410095
1999 0.5639185309410095
Saving
1 0.6924312710762024
501 0.564057469367981
1001 0.5637767314910889
1501 0.5637767910957336
1999 0.5637767910957336
Saving
Finished 4197 609682763438-rest
1 0.6255918145179749
501 0.4884607791900635
1001 0.4854710102081299
1501 0.4826095998287201
1999 0.4798301160335541
Saving
1 0.6269665956497192
501 0.48609602451324463
1001 0.4778497815132141
1501 0.4778440296649933
1999 0.4778440296649933
Saving
Finished 4198 609682763438-nback
1 0.591346800327301
501 0.48493820428848267
1001 0.4794086217880249
1501 0.4794085919857025
1999 0.4794085919857025
Saving
1 0.6109983325004578
501 0.4903568923473358
1001 0.48533502221107483
1501 0.48517173528671265
1999 0.4851699769496918
Finished 4199 609682763438-emoid
1 0.6469699740409851
501 0.5661944

501 0.4686288833618164
1001 0.4672391414642334
1501 0.4672311246395111
1999 0.4672310948371887
Saving
1 0.594836950302124
501 0.47237706184387207
1001 0.4713784456253052
1501 0.4709622859954834
1999 0.47092992067337036
Finished 4226 609728145608-emoid
1 0.6574005484580994
501 0.3521217107772827
1001 0.3511792719364166
1501 0.3501029908657074
1999 0.35009393095970154
Saving
1 0.6578919887542725
501 0.34963086247444153
1001 0.3458308279514313
1501 0.34240588545799255
1999 0.34239906072616577
Saving
Finished 4227 609728667591-nback
1 0.6099228858947754
501 0.3814847767353058
1001 0.3814659118652344
1501 0.38146594166755676
1999 0.38146594166755676
Saving
1 0.6353641748428345
501 0.4397742450237274
1001 0.4128572344779968
1501 0.38716498017311096
1999 0.38716498017311096
Finished 4228 609728667591-emoid
1 0.6151332855224609
501 0.29527756571769714
1001 0.29335010051727295
1501 0.2930150628089905
1999 0.2930150628089905
Saving
1 0.5973764657974243
501 0.2836138904094696
1001 0.2836072444915

1501 0.5564851760864258
1999 0.5564852356910706
Saving
Finished 4255 609781172292-emoid
1 0.622002363204956
501 0.48294052481651306
1001 0.4652705490589142
1501 0.4641473591327667
1999 0.4639841616153717
Saving
1 0.6216612458229065
501 0.489946573972702
1001 0.4610249102115631
1501 0.4607767164707184
1999 0.4607739746570587
Saving
Finished 4256 609788818607-rest
1 0.5662683844566345
501 0.32716819643974304
1001 0.32651737332344055
1501 0.3265165388584137
1999 0.3265165388584137
Saving
1 0.6043285131454468
501 0.32773876190185547
1001 0.3272647261619568
1501 0.32706162333488464
1999 0.3268239200115204
Finished 4257 609788818607-nback
1 0.582623302936554
501 0.37065359950065613
1001 0.36478668451309204
1501 0.36243781447410583
1999 0.36196738481521606
Saving
1 0.5998207330703735
501 0.37815409898757935
1001 0.3688020408153534
1501 0.36091896891593933
1999 0.3603310286998749
Saving
Finished 4258 609788818607-emoid
1 0.6117424368858337
501 0.4171323776245117
1001 0.41388753056526184
1501 0

501 0.548099160194397
1001 0.545111358165741
1501 0.5451112389564514
1999 0.5451112389564514
Saving
Finished 4285 609847053312-emoid
1 0.6511019468307495
501 0.48728078603744507
1001 0.4872804582118988
1501 0.4872804582118988
1999 0.48486095666885376
Saving
1 0.6358758807182312
501 0.491914302110672
1001 0.49191418290138245
1501 0.49191418290138245
1999 0.49191418290138245
Finished 4286 609849166759-rest
1 0.6682383418083191
501 0.5748202800750732
1001 0.5693987607955933
1501 0.5691078901290894
1999 0.5691078901290894
Saving
1 0.6562753319740295
501 0.5678835511207581
1001 0.5672280788421631
1501 0.5664185881614685
1999 0.5664185881614685
Saving
Finished 4287 609849166759-nback
1 0.6382362246513367
501 0.5324354767799377
1001 0.5273937582969666
1501 0.5273934602737427
1999 0.5273934602737427
Saving
1 0.6397923231124878
501 0.5355833172798157
1001 0.5281557440757751
1501 0.5251675248146057
1999 0.5251675248146057
Saving
Finished 4288 609849166759-emoid
1 0.6396001577377319
501 0.4090398

1001 0.5366095304489136
1501 0.536429226398468
1999 0.5364291667938232
Saving
1 0.6391810178756714
501 0.5450817346572876
1001 0.5426012873649597
1501 0.5426013469696045
1999 0.5426013469696045
Finished 4315 609937840506-emoid
1 0.6046250462532043
501 0.3849550783634186
1001 0.37665247917175293
1501 0.3766283392906189
1999 0.3766283094882965
Saving
1 0.6331837773323059
501 0.3795895576477051
1001 0.3748961091041565
1501 0.3748961091041565
1999 0.3748961091041565
Saving
Finished 4316 609950120523-rest
1 0.6391659379005432
501 0.5363285541534424
1001 0.5358564257621765
1501 0.5358564257621765
1999 0.5358564257621765
Saving
1 0.6509958505630493
501 0.5365102887153625
1001 0.5350367426872253
1501 0.5350367426872253
1999 0.5350367426872253
Saving
Finished 4317 609950120523-nback
1 0.6649828553199768
501 0.5007081627845764
1001 0.4990437924861908
1501 0.4990437924861908
1999 0.4990437924861908
Saving
1 0.6690687537193298
501 0.49912771582603455
1001 0.4991258382797241
1501 0.4991258382797241

In [22]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisNoJit1.pkl', 'wb'))

print('Complete')

Complete
